In [2]:
import numpy as np
import pandas as pd
import sklearn as sk
import pandas as pd
import seaborn as sns
import keras
from keras import Sequential
from keras.layers import Dense
import matplotlib.pyplot as plt
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score



d = pd.read_csv('houses_edited.csv')


features = ["bedrooms", "bathrooms", "sqft", "parking", 
            "mean_district_income", "Plex", "Semi-Detached", 
            "Store W/Apt/Offc", "Co-Op Apt", "Co-Ownership Apt", 
            "Link", "Comm Element Condo", "Att/Row/Twnhouse", 
            "Condo Townhouse", "Detached", "Condo Apt"] #define list of features to input in nerual network


df = pd.DataFrame(data = d)


def remove_beds(x): #function to remove the word "beds" from column of number of beds
    x = x.replace(" beds", "")
    return x
def calc_beds(j): #function to calculate number of beds from string
    answer = sum(int(i) for i in j.split('+'))
    return answer


df.bedrooms = df.bedrooms.apply(remove_beds) 
df.bedrooms = df.bedrooms.apply(calc_beds)

In [3]:
df = df.drop(["description", "mls", "bedrooms_ag", "bedrooms_bg", "full_link", "full_address", "title",
              "final_price_transformed", "final_price_log", "city_district", "district_code",
              "index", "lat", "long", "list_price"], axis = 1) #Remove insignificant values

#Fill missing sqft values based on type of property

sqft_Store = 0
sqft_Link = 0
sqft_Twn = 0
sqft_Plex = 0
sqft_Detached = 0
sqft_Semi_detached = 0
sqft_Store_count = 0
sqft_Link_count = 0
sqft_Twn_count = 0
sqft_Plex_count = 0
sqft_Detached_count = 0
sqft_Semi_detached_count = 0


for index, row in df.iterrows():
    if pd.notnull(row["sqft"]) and row["type"] == "Store W/Apt/Offc":
      sqft_Store += row["sqft"]
      sqft_Store_count += 1
    if pd.notnull(row["sqft"]) and row["type"] == "Link":
      sqft_Link += row["sqft"]
      sqft_Link_count += 1
    if pd.notnull(row["sqft"]) and row["type"] == "Att/Row/Twnhouse":
      sqft_Twn += row["sqft"]
      sqft_Twn_count += 1
    if pd.notnull(row["sqft"]) and row["type"] == "Plex":
      sqft_Plex += row["sqft"]
      sqft_Plex_count += 1
    if pd.notnull(row["sqft"]) and row["type"] == "Detached":
      sqft_Detached += row["sqft"]
      sqft_Detached_count += 1
    if pd.notnull(row["sqft"]) and row["type"] == "Semi-Detached":
      sqft_Semi_detached += row["sqft"]
      sqft_Semi_detached_count += 1

avg_Store = sqft_Store/sqft_Store_count
avg_Link = sqft_Link/sqft_Link_count
avg_Twn = sqft_Twn/sqft_Twn_count
avg_Plex = sqft_Plex/sqft_Plex_count
avg_Detached = sqft_Detached/sqft_Detached_count
avg_Semi_detached = sqft_Semi_detached/sqft_Semi_detached_count


for index, row in df.iterrows():
  #print(row["type"], row["sqft"])
  if pd.isnull(row["sqft"]) and row["type"] == "Store W/Apt/Offc":
    df.at[index, "sqft"] = avg_Store

  if pd.isnull(row["sqft"]) and row["type"] == "Link":
    df.at[index, "sqft"] = avg_Link

  if pd.isnull(row["sqft"]) and row["type"] == "Att/Row/Twnhouse":
    df.at[index, "sqft"] = avg_Twn

  if pd.isnull(row["sqft"]) and row["type"] == "Plex":
    df.at[index, "sqft"] = avg_Plex

  if pd.isnull(row["sqft"]) and row["type"] == "Detached":
    df.at[index, "sqft"] = avg_Detached

  if pd.isnull(row["sqft"]) and row["type"] == "Semi-Detached":
    df.at[index, "sqft"] = avg_Semi_detached


dummy_type = pd.get_dummies(df["type"]) #allow 
df = pd.concat([df, dummy_type], axis = 1)
df = df.drop(["type"], axis = 1)

In [5]:
train, test = train_test_split(df, test_size = 0.20, random_state = 0) #Split data into 2 parts, 
                                                                       #training and testing data, with ratio 8:2 

x_train = train[features]
y_train = train[['final_price']]

model = Sequential()

model.add(Dense(64, input_dim = 16,activation = 'relu'))
model.add(Dense(32, activation = 'relu'))
model.add(Dense(1, activation = 'relu'))

opt = keras.optimizers.Adam(learning_rate = 0.03) 

model.compile(loss='mean_squared_error', optimizer=opt, metrics=['MeanAbsolutePercentageError'])
model.fit(x_train, y_train, epochs = 1900, batch_size = 200, shuffle = False, verbose = 2)
#training = model.predict(x_train)
_, accuracy = model.evaluate(x_train, y_train)
print('Accuracy: %.2f' % (accuracy))

predictions = model.predict(x_train)
#print(predictions)
for i in range(10):
  print("%d, expected: %d" % (predictions[i], y_train.iloc[i]))

# Predicting prices from test set
# x_test = test[features]
# y_test = test[['final_price']]


# print(model.evaluate(x_test, y_test))


#Best hyperparameter result
# MSPE: 17.95%
# Neural layers: 64, 32, 1
# Learning rate: 0.03
# Epochs: 1500
# Batch size; 200


Epoch 1/1900
61/61 - 0s - loss: 351973081088.0000 - mean_absolute_percentage_error: 41.2487
Epoch 2/1900
61/61 - 0s - loss: 201166880768.0000 - mean_absolute_percentage_error: 29.2885
Epoch 3/1900
61/61 - 0s - loss: 171798593536.0000 - mean_absolute_percentage_error: 28.3934
Epoch 4/1900
61/61 - 0s - loss: 167540293632.0000 - mean_absolute_percentage_error: 28.4107
Epoch 5/1900
61/61 - 0s - loss: 164673126400.0000 - mean_absolute_percentage_error: 28.5623
Epoch 6/1900
61/61 - 0s - loss: 163876683776.0000 - mean_absolute_percentage_error: 28.7553
Epoch 7/1900
61/61 - 0s - loss: 162886041600.0000 - mean_absolute_percentage_error: 28.7067
Epoch 8/1900
61/61 - 0s - loss: 163414704128.0000 - mean_absolute_percentage_error: 28.8230
Epoch 9/1900
61/61 - 0s - loss: 165476040704.0000 - mean_absolute_percentage_error: 29.0540
Epoch 10/1900
61/61 - 0s - loss: 164972478464.0000 - mean_absolute_percentage_error: 29.0780
Epoch 11/1900
61/61 - 0s - loss: 165859590144.0000 - mean_absolute_percentage_e

Epoch 90/1900
61/61 - 0s - loss: 158047453184.0000 - mean_absolute_percentage_error: 28.0032
Epoch 91/1900
61/61 - 0s - loss: 157863460864.0000 - mean_absolute_percentage_error: 27.9836
Epoch 92/1900
61/61 - 0s - loss: 157733928960.0000 - mean_absolute_percentage_error: 27.9804
Epoch 93/1900
61/61 - 0s - loss: 157852860416.0000 - mean_absolute_percentage_error: 27.9774
Epoch 94/1900
61/61 - 0s - loss: 157787586560.0000 - mean_absolute_percentage_error: 27.9828
Epoch 95/1900
61/61 - 0s - loss: 157868490752.0000 - mean_absolute_percentage_error: 27.9990
Epoch 96/1900
61/61 - 0s - loss: 157750771712.0000 - mean_absolute_percentage_error: 27.9506
Epoch 97/1900
61/61 - 0s - loss: 157498753024.0000 - mean_absolute_percentage_error: 27.9538
Epoch 98/1900
61/61 - 0s - loss: 157542694912.0000 - mean_absolute_percentage_error: 27.9177
Epoch 99/1900
61/61 - 0s - loss: 157401464832.0000 - mean_absolute_percentage_error: 27.9345
Epoch 100/1900
61/61 - 0s - loss: 157661478912.0000 - mean_absolute_pe

Epoch 178/1900
61/61 - 0s - loss: 151668998144.0000 - mean_absolute_percentage_error: 27.1137
Epoch 179/1900
61/61 - 0s - loss: 151438131200.0000 - mean_absolute_percentage_error: 27.0695
Epoch 180/1900
61/61 - 0s - loss: 151549804544.0000 - mean_absolute_percentage_error: 27.0390
Epoch 181/1900
61/61 - 0s - loss: 151245012992.0000 - mean_absolute_percentage_error: 27.1041
Epoch 182/1900
61/61 - 0s - loss: 151186472960.0000 - mean_absolute_percentage_error: 27.1137
Epoch 183/1900
61/61 - 0s - loss: 150706962432.0000 - mean_absolute_percentage_error: 26.9401
Epoch 184/1900
61/61 - 0s - loss: 150314631168.0000 - mean_absolute_percentage_error: 26.9386
Epoch 185/1900
61/61 - 0s - loss: 150186459136.0000 - mean_absolute_percentage_error: 26.9556
Epoch 186/1900
61/61 - 0s - loss: 149782052864.0000 - mean_absolute_percentage_error: 26.9835
Epoch 187/1900
61/61 - 0s - loss: 149464006656.0000 - mean_absolute_percentage_error: 26.9182
Epoch 188/1900
61/61 - 0s - loss: 148611153920.0000 - mean_a

61/61 - 0s - loss: 127943106560.0000 - mean_absolute_percentage_error: 22.9536
Epoch 266/1900
61/61 - 0s - loss: 127884869632.0000 - mean_absolute_percentage_error: 22.6442
Epoch 267/1900
61/61 - 0s - loss: 128547799040.0000 - mean_absolute_percentage_error: 22.6748
Epoch 268/1900
61/61 - 0s - loss: 127763988480.0000 - mean_absolute_percentage_error: 23.0204
Epoch 269/1900
61/61 - 0s - loss: 126569160704.0000 - mean_absolute_percentage_error: 22.7285
Epoch 270/1900
61/61 - 0s - loss: 127129296896.0000 - mean_absolute_percentage_error: 22.6303
Epoch 271/1900
61/61 - 0s - loss: 126866735104.0000 - mean_absolute_percentage_error: 22.4486
Epoch 272/1900
61/61 - 0s - loss: 126979809280.0000 - mean_absolute_percentage_error: 22.6389
Epoch 273/1900
61/61 - 0s - loss: 127721177088.0000 - mean_absolute_percentage_error: 22.7087
Epoch 274/1900
61/61 - 0s - loss: 127844204544.0000 - mean_absolute_percentage_error: 22.6444
Epoch 275/1900
61/61 - 0s - loss: 128129892352.0000 - mean_absolute_percent

Epoch 353/1900
61/61 - 0s - loss: 124889317376.0000 - mean_absolute_percentage_error: 22.8027
Epoch 354/1900
61/61 - 0s - loss: 124505448448.0000 - mean_absolute_percentage_error: 22.6697
Epoch 355/1900
61/61 - 0s - loss: 125067255808.0000 - mean_absolute_percentage_error: 22.7769
Epoch 356/1900
61/61 - 0s - loss: 124606840832.0000 - mean_absolute_percentage_error: 22.4444
Epoch 357/1900
61/61 - 0s - loss: 124296192000.0000 - mean_absolute_percentage_error: 22.2823
Epoch 358/1900
61/61 - 0s - loss: 124421251072.0000 - mean_absolute_percentage_error: 22.5492
Epoch 359/1900
61/61 - 0s - loss: 122516791296.0000 - mean_absolute_percentage_error: 21.8834
Epoch 360/1900
61/61 - 0s - loss: 125330833408.0000 - mean_absolute_percentage_error: 22.7817
Epoch 361/1900
61/61 - 0s - loss: 124002410496.0000 - mean_absolute_percentage_error: 22.5893
Epoch 362/1900
61/61 - 0s - loss: 125794205696.0000 - mean_absolute_percentage_error: 22.7801
Epoch 363/1900
61/61 - 0s - loss: 122713497600.0000 - mean_a

...

Epoch 1487/1900
61/61 - 0s - loss: 116140990464.0000 - mean_absolute_percentage_error: 21.1660
Epoch 1488/1900
61/61 - 0s - loss: 117185486848.0000 - mean_absolute_percentage_error: 21.3843
Epoch 1489/1900
61/61 - 0s - loss: 112223453184.0000 - mean_absolute_percentage_error: 20.4492
Epoch 1490/1900
61/61 - 0s - loss: 113236328448.0000 - mean_absolute_percentage_error: 20.5359
Epoch 1491/1900
61/61 - 0s - loss: 116564410368.0000 - mean_absolute_percentage_error: 21.3215
Epoch 1492/1900
61/61 - 0s - loss: 113298030592.0000 - mean_absolute_percentage_error: 20.7829
Epoch 1493/1900
61/61 - 0s - loss: 111917850624.0000 - mean_absolute_percentage_error: 20.3759
Epoch 1494/1900
61/61 - 0s - loss: 115567861760.0000 - mean_absolute_percentage_error: 21.0711
Epoch 1495/1900
61/61 - 0s - loss: 115508764672.0000 - mean_absolute_percentage_error: 21.1547
Epoch 1496/1900
61/61 - 0s - loss: 111505219584.0000 - mean_absolute_percentage_error: 20.4050
Epoch 1497/1900
61/61 - 0s - loss: 111603646464.00

Epoch 1574/1900
61/61 - 0s - loss: 115057729536.0000 - mean_absolute_percentage_error: 21.0634
Epoch 1575/1900
61/61 - 0s - loss: 112474300416.0000 - mean_absolute_percentage_error: 20.4977
Epoch 1576/1900
61/61 - 0s - loss: 117456191488.0000 - mean_absolute_percentage_error: 21.7475
Epoch 1577/1900
61/61 - 0s - loss: 112093847552.0000 - mean_absolute_percentage_error: 20.6785
Epoch 1578/1900
61/61 - 0s - loss: 115363438592.0000 - mean_absolute_percentage_error: 21.1500
Epoch 1579/1900
61/61 - 0s - loss: 110932500480.0000 - mean_absolute_percentage_error: 20.4050
Epoch 1580/1900
61/61 - 0s - loss: 111739912192.0000 - mean_absolute_percentage_error: 20.4745
Epoch 1581/1900
61/61 - 0s - loss: 115029508096.0000 - mean_absolute_percentage_error: 20.7718
Epoch 1582/1900
61/61 - 0s - loss: 117902868480.0000 - mean_absolute_percentage_error: 21.8233
Epoch 1583/1900
61/61 - 0s - loss: 109852975104.0000 - mean_absolute_percentage_error: 20.3450
Epoch 1584/1900
61/61 - 0s - loss: 115302113280.00

Epoch 1661/1900
61/61 - 0s - loss: 114514526208.0000 - mean_absolute_percentage_error: 21.1847
Epoch 1662/1900
61/61 - 0s - loss: 111298519040.0000 - mean_absolute_percentage_error: 20.6419
Epoch 1663/1900
61/61 - 0s - loss: 113954955264.0000 - mean_absolute_percentage_error: 21.1435
Epoch 1664/1900
61/61 - 0s - loss: 109967245312.0000 - mean_absolute_percentage_error: 20.4830
Epoch 1665/1900
61/61 - 0s - loss: 109632028672.0000 - mean_absolute_percentage_error: 20.2172
Epoch 1666/1900
61/61 - 0s - loss: 124461826048.0000 - mean_absolute_percentage_error: 22.6842
Epoch 1667/1900
61/61 - 0s - loss: 118804119552.0000 - mean_absolute_percentage_error: 22.1440
Epoch 1668/1900
61/61 - 0s - loss: 111471190016.0000 - mean_absolute_percentage_error: 20.7103
Epoch 1669/1900
61/61 - 0s - loss: 111844466688.0000 - mean_absolute_percentage_error: 20.9136
Epoch 1670/1900
61/61 - 0s - loss: 110084513792.0000 - mean_absolute_percentage_error: 20.3630
Epoch 1671/1900
61/61 - 0s - loss: 112832299008.00

Epoch 1748/1900
61/61 - 0s - loss: 123392401408.0000 - mean_absolute_percentage_error: 22.8254
Epoch 1749/1900
61/61 - 0s - loss: 117842944000.0000 - mean_absolute_percentage_error: 22.1231
Epoch 1750/1900
61/61 - 0s - loss: 110354284544.0000 - mean_absolute_percentage_error: 20.8117
Epoch 1751/1900
61/61 - 0s - loss: 109937311744.0000 - mean_absolute_percentage_error: 20.4088
Epoch 1752/1900
61/61 - 0s - loss: 118355705856.0000 - mean_absolute_percentage_error: 21.6584
Epoch 1753/1900
61/61 - 0s - loss: 112052297728.0000 - mean_absolute_percentage_error: 21.2336
Epoch 1754/1900
61/61 - 0s - loss: 113054457856.0000 - mean_absolute_percentage_error: 21.1004
Epoch 1755/1900
61/61 - 0s - loss: 113928560640.0000 - mean_absolute_percentage_error: 21.0902
Epoch 1756/1900
61/61 - 0s - loss: 113194156032.0000 - mean_absolute_percentage_error: 21.0046
Epoch 1757/1900
61/61 - 0s - loss: 113528061952.0000 - mean_absolute_percentage_error: 21.1269
Epoch 1758/1900
61/61 - 0s - loss: 110550122496.00

Epoch 1835/1900
61/61 - 0s - loss: 113547304960.0000 - mean_absolute_percentage_error: 21.2481
Epoch 1836/1900
61/61 - 0s - loss: 113285611520.0000 - mean_absolute_percentage_error: 21.2228
Epoch 1837/1900
61/61 - 0s - loss: 112942669824.0000 - mean_absolute_percentage_error: 21.2599
Epoch 1838/1900
61/61 - 0s - loss: 117615345664.0000 - mean_absolute_percentage_error: 21.6272
Epoch 1839/1900
61/61 - 0s - loss: 110478950400.0000 - mean_absolute_percentage_error: 20.9709
Epoch 1840/1900
61/61 - 0s - loss: 112298000384.0000 - mean_absolute_percentage_error: 21.1051
Epoch 1841/1900
61/61 - 0s - loss: 112267001856.0000 - mean_absolute_percentage_error: 21.1639
Epoch 1842/1900
61/61 - 0s - loss: 109196681216.0000 - mean_absolute_percentage_error: 20.6087
Epoch 1843/1900
61/61 - 0s - loss: 108448620544.0000 - mean_absolute_percentage_error: 20.3640
Epoch 1844/1900
61/61 - 0s - loss: 121257926656.0000 - mean_absolute_percentage_error: 22.0447
Epoch 1845/1900
61/61 - 0s - loss: 113320624128.00